# The Premise

Remember this image? Well, each part interacts with each other, but fundamentally can be written into individual python scripts for improved organization

<img src="Workflow.png" alt="The WorkFlow" width="750">

So, no new code in this section! Just pure organization and separating all the parts of our machine learning process from the last part, custom datasets

<img src="Division.png" alt="The WorkFlow" width="750">

So in the end, we can just use a 1 liner like this, and directly make a model and train it with these hyper parameters

<img src="Command Line.png" alt="The WorkFlow" width="750">

It's how the real folks do it in the industry, they divide their python files to take different parts in this machine learning madness

It's also good for the reusability of the code, since as scripts, we can just drop and play around with them on different models

<img src="TorchVision.png" alt="The WorkFlow" width="750">

# Logical Divsion

After we get the logic of the code down, we will chuck all the essential code (from custom datasets) into python scripts

## Data

Overview:

1. Getting Data
    - Download the data with (`Requests`) then unzip it with (`Zipfile`)
    - Set up a training data directory, and testing data directory and split data 80/20 as (`Training/Testing)

2. Loading Data
    - Load the data with torchvision's default (`Dataset`) and (`Dataloader`), 1 training 1 testing

## Model

Overview:

1. Make Model
    - Define the model class with (`nn.Module`), write the (`_init_`) and (`forward`) function

2. Train/Test Model
    - Define the (`Loss Function`) and (`Optimizer`)
    - Write out a (`training_step`), and a (`testing step`)
    - Write out the (`training loop`)

3. Save/Load Model
    - Define a function that saves a PyTorch model to a model foler, with (`Pathlib`)
    - Also, another function that loads a PyTorch model, for usage or further training

4. Use Model
    - Define a function that performs a forward pass on an existing, trained Pytorch model

# The Data

Will be different in detail of implementation as python scripts, but the essence is the same 

## Getting Data

In [ ]:
#The library that allows us to interact with web services and APIs
import requests

#As the name suggests... allows us to unzip or zip files
import zipfile

#Setting up directories, folders, getting things to places
from pathlib import Path

#For Deleting the Zip File After Download
import os


#Setup path to a folder
data_path = Path("Data/")


#Creating data folder
if data_path.is_dir():
    print(f"{data_path} directory already exists, skipping download")
else:
    print(f"{data_path} does not exist, creating directory")
    data_path.mkdir(parents=True, exist_ok=True)


    #Downloading Data Zip file, wb means write binary
    with open(data_path/"pizza_steak_sushi.zip","wb") as a:
        request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
        print("Downloading image data...")
        a.write(request.content)
    
    
    #Unzipping Files in Zip
    with zipfile.ZipFile(data_path/"pizza_steak_sushi.zip", "r") as images:
        print("Unzipping Image Data")
        images.extractall(data_path)


    # Delete the zip file
    zip_file_path = data_path / "pizza_steak_sushi.zip"
    try:
        os.remove(zip_file_path)
        print(f"{zip_file_path} deleted successfully.")
    except OSError as e:
        print(f"Error: {e}")

## Loading Data

In [ ]:
# Use ImageFolder to create dataset(s)
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

train_dir = data_path/"train"
test_dir = data_path/"test"


#This step describes the transformations of images before turning into tensors
data_transform = transforms.Compose([

    #Change the size
    transforms.Resize(size=(128,128)),

    #Random application of a shit ton of jizz jazz
    transforms.TrivialAugmentWide(num_magnitude_bins=31),

    #Turns the image into a tensor
    transforms.ToTensor()
])


#target folder of images, and transforms to perform on data (images)
train_data = datasets.ImageFolder(root=train_dir, transform=data_transform) 
test_data = datasets.ImageFolder(root=test_dir, transform=data_transform)

#batch size is how many samples in a batch
#num workers is how many cores you want the cpu to be running this dataloader on
train_dataloader = DataLoader(dataset=train_data, batch_size=5, num_workers=0, shuffle=True)
test_dataloader = DataLoader(dataset=test_data, batch_size=1, num_workers=0, shuffle=False)

# The Model

Will be different in detail of implementation as python scripts, but the essence is the same 

## Making Model

In [ ]:
import torch
from torch import nn

device = "cuda" if torch.cuda.is_available() else "cpu"



class TinyVGG(nn.Module):


    def __init__(self):

        """
        Replicates the TinyVGG architecture from the CNN explainer website in PyTorch.
        See the original architecture here: https://poloclub.github.io/cnn-explainer/

        There are no parameters that you can alter here for creating the neural network

        It is defined as 3 consecutive 2D convolutional blocks, each block with (Conv2d, LeakyRelu, Conv2d, LeakyRelu, MaxPool2d)
        In channels = 10, Out channels = 10, kernel_size = 3, stride = 1, padding = 1

        Then a sequential layer with (Flatten, Linear, Softmax)
        """

        super().__init__()

        #The in channels is no longer 1, as now we have 3 channels RGB from our input
        self.conv_b1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=10, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(),
            nn.Conv2d(in_channels=10, out_channels=10, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.conv_b2 = nn.Sequential(
            nn.Conv2d(in_channels=10, out_channels=10, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(),
            nn.Conv2d(in_channels=10, out_channels=10, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.conv_b3 = nn.Sequential(
            nn.Conv2d(in_channels=10, out_channels=10, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(),
            nn.Conv2d(in_channels=10, out_channels=10, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        #again, we don't know the input feature size, so we just run and see the tensor error, then replace it with the correct number
        self.end_classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=2560, out_features=3), 
            nn.Softmax(dim=1)
        )

    #ok... it's not really just a image, when through all those convolution layers there's like 10 "feature" images running parallel
    def forward(self, image):
        return self.end_classifier(self.conv_b3(self.conv_b2(self.conv_b1(image))))

## Training/Testing

In [ ]:
from tqdm.auto import tqdm


def train_step(model, data_loader, loss_fn, optimizer, device, epoch):

    model.train()
    train_loss, train_acc = 0, 0

    for batch, (image, label) in enumerate(data_loader):

        image, label = image.to(device), label.to(device)
        prediction = model(image)

        loss = loss_fn(prediction, label)
        train_loss += loss.item()

        prediction_label = torch.argmax(prediction, dim=1)
        train_acc += (prediction_label == label).sum().item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    train_loss /= len(data_loader)
    train_acc /= len(data_loader)
    print(f"Epoch: {epoch + 1} | Train Loss: {train_loss:.5f} | Train Accuracy: {train_acc:.2f}%")



def test_step(model, data_loader, loss_fn, device, epoch):

    model.eval()
    test_loss, test_acc = 0, 0

    with torch.inference_mode():
        for batch, (image, label) in enumerate(data_loader):

           image, label = image.to(device), label.to(device)
           prediction = model(image)

           loss = loss_fn(prediction, label)
           test_loss += loss.item()

           prediction_label = torch.argmax(prediction, dim=1)
           test_acc += (prediction_label == label).sum().item()

    test_loss /= len(data_loader)
    test_acc /= len(data_loader)
    print(f"Epoch: {epoch + 1} | Test Loss: {test_loss:.5f} | Test Accuracy: {test_acc:.2f}%")



def train(model, train, test, loss_fn, optimizer, device, epochs):
    for epoch in tqdm(range(epochs)):
        train_step(model, train, loss_fn, optimizer, device, epoch)
        test_step(model, test, loss_fn, device, epoch)

## Saving/Loading

In [ ]:
from pathlib import Path


def save_model(model, model_name):

    #Create directory
    model_path = Path("models")
    model_path.mkdir(parents=True, exist_ok=True)

    #Create saving path, usually pytorch files are called "pth"
    model_name = model_name
    model_save_path = model_path / model_name

    #Saving the state dict
    print(f"Saving model to: {model_save_path}")
    torch.save(obj=model.state_dict(), f=model_save_path)



def load_model(model_class, model_name):

    #we'll need to create a new model and load the saved state_dict() into the new model
    model = model_class()

    #loading the saved state dict from the new model, with torch.load()
    model.load_state_dict(torch.load(f=model_name))

    return model